In [ ]:
if not os.path.isfile("estrutura_ud.py"):
    ! wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py
import estrutura_ud

In [ ]:
file = "../Petroles_3/Petroles_3_golden.conllu"
corpus = estrutura_ud.Corpus(recursivo=True)
corpus.load(file)

for sentence in corpus.sentences.values():
    for token in sentence.tokens:
        if token.word.lower() in ["fácie", "fácies"]:
            token.lemma = "fácies"
            if token.upos == "ADJ":
                token.upos = "NOUN"
                token.deprel = token.head_token.deprel
                token.dephead = token.head_token.dephead
                token.head_token.dephead = token.id
                token.head_token.upos = "ADJ"
                token.head_token.deprel = "amod"
                for _token in sentence.tokens:
                    if _token.dephead == token.head_token.id:
                        _token.dephead = token.id
    
    for token in sentence.tokens:
        if token.deprel == "det" and token.head_token.upos not in ["VERB", "NUM"]:
            if 'PronType=Art' in token.feats or 'PronType=Dem' in token.feats:
                for feat in ["Gender=", "Number="]:
                    if feat in token.feats:
                        token.head_token.feats = "|".join(sorted(
                            [x for x in token.head_token.feats.split("|") if not x.startswith(feat) and x != "_"] + 
                            ["{}{}".format(
                                feat,
                                token.feats.split(feat)[1].split("|")[0]
                                )]))
                    elif feat in token.head_token.feats:
                        token.feats = "|".join(sorted(
                            [x for x in token.feats.split("|") if not x.startswith(feat) and x != "_"] + 
                            ["{}{}".format(
                                feat,
                                token.head_token.feats.split(feat)[1].split("|")[0]
                                )]))
            else:
                for feat in ["Gender=", "Number="]:
                    if feat in token.head_token.feats:
                        token.feats = "|".join(sorted(
                            [x for x in token.feats.split("|") if not x.startswith(feat) and x != "_"] + 
                            ["{}{}".format(
                                feat,
                                token.head_token.feats.split(feat)[1].split("|")[0]
                                )]))
                    elif feat in token.feats:
                        token.head_token.feats = "|".join(sorted(
                            [x for x in token.head_token.feats.split("|") if not x.startswith(feat) and x != "_"] + 
                            ["{}{}".format(
                                feat,
                                token.feats.split(feat)[1].split("|")[0]
                                )]))
            if not token.feats:
                token.feats = "_"
            if not token.head_token.feats:
                token.head_token.feats = "_"

file_name = file.rsplit("/", 1)[1]
corpus.save("regras.conllu")
os.system("cp {} antes-{}".format(file, file_name))
os.system("rm regras-correcao.zip; zip regras-correcao.zip antes-{} regras.conllu".format(file_name))
os.remove("antes-{}".format(file_name))
os.system("python3 ../../conllu-merge-resolver/cosmo.py {} regras.conllu '.*'".format(file))
os.remove("regras.conllu")